In [12]:
import pandas as pd
import matplotlib.pyplot as plt
# from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
pd.set_option('display.max_columns', None)
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize as wt
set(stopwords.words('english'))


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [4]:
# Galvanize item recommender class
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

class ItemRecommender():
    '''
    Content based item recommender
    '''
    def __init__(self, similarity_measure=None):
        self.similarity_matrix = None
        self.item_names = None

        if similarity_measure == None:
            self.similarity_measure = cosine_similarity
        else:
            self.similarity_measure = similarity_measure

    
    def fit(self, X, index=None):
        '''
        Takes a numpy array of the item attributes and creates the similarity matrix
        INPUT -
            X: NUMPY ARRAY - Rows are items, columns are feature values / or DF
            index: LIST - List of the item names/titles in order of the numpy arrray
        
        OUTPUT - None
        Notes:  You might want to keep titles and X as attributes to refer to them later
        Create the a similarity matrix of item to item similarity
        '''

        # While keeping this as a sparse matrix would be best the cosign sim
        # function returns a array so there is no reason.
        
        if isinstance(X, pd.DataFrame):
            self.item_counts = X
            self.item_names = X.index
            self.similarity_df = pd.DataFrame(self.similarity_measure(X.values, X.values),
                 index = self.item_names)
        else:
            self.item_counts = X
            self.similarity_df = pd.DataFrame(self.similarity_measure(X, X),
                 index = index)
            self.item_names = self.similarity_df.index

        
    def get_recommendations(self, item, n=5):
        '''
        Returns the top n items related to the item passed in
        INPUT:
            item    - STRING - Name of item in the original DataFrame 
            n       - INT    - Number of top related items to return 
        OUTPUT:
            items - List of the top n related item names
        For a given item find the n most similar items to it (this can be done using the similarity matrix created in the fit method)
        '''
        return self.item_names[self.similarity_df.loc[item].values.argsort()[-(n+1):-1]].values[::-1]


    def get_user_profile(self, items):
        '''
        Takes a list of items and returns a user profile. A vector representing the likes of the user.
        INPUT: 
            items  -   LIST - list of movie names user likes / has seen
        OUTPUT: 
            user_profile - NP ARRAY - array representing the likes of the user 
                    The columns of this will match the columns of the trained on matrix
    
        Using the list of items liked by the user create a profile which will be a 1 x number of features array.  This should be the addition of the values for all liked item features (you can choose how to normalize if you think it is needed)
        '''
        user_profile = np.zeros(self.item_counts.shape[1])
        for item in items:
            user_profile += self.item_counts.loc[item].values

        return user_profile


    def get_user_recommendation(self, items, n=5):
        '''
        Takes a list of movies user liked and returns the top n items for that user
        INPUT 
            items  -   LIST - list of movie names user likes / has seen
            n -  INT - number of items to return
        OUTPUT 
            items - LIST - n recommended items
    
        Make use of the get_user_profile method to create a user profile that will be used to get the similarity to all items and recommend the top n.
        '''
        num_items = len(items)
        user_profile = self.get_user_profile(items)

        user_sim =  self.similarity_measure(self.item_counts, user_profile.reshape(1,-1))

        return self.item_names[user_sim[:,0].argsort()[-(num_items+n):-num_items]].values[::-1]





Attempt 1

In [5]:
anime = pd.read_csv('animelist.csv')

In [6]:
anime.columns 

Index(['anime_id', 'title', 'title_english', 'title_japanese',
       'title_synonyms', 'image_url', 'type', 'source', 'episodes', 'status',
       'airing', 'aired_string', 'aired', 'duration', 'rating', 'score',
       'scored_by', 'rank', 'popularity', 'members', 'favorites', 'background',
       'premiered', 'broadcast', 'related', 'producer', 'licensor', 'studio',
       'genre', 'opening_theme', 'ending_theme'],
      dtype='object')

In [7]:
anime = anime[['title','title_english','title_japanese','title_synonyms', 'type', 'source', 'status','airing', 'rating', 'background', 'producer', 'licensor', 'studio','genre','opening_theme','ending_theme']]


In [8]:
anime = anime.dropna(axis = 0, subset = ['title'])
anime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14478 entries, 0 to 14477
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           14478 non-null  object
 1   title_english   5724 non-null   object
 2   title_japanese  14443 non-null  object
 3   title_synonyms  8937 non-null   object
 4   type            14478 non-null  object
 5   source          14478 non-null  object
 6   status          14478 non-null  object
 7   airing          14478 non-null  bool  
 8   rating          14478 non-null  object
 9   background      1057 non-null   object
 10  producer        8288 non-null   object
 11  licensor        3373 non-null   object
 12  studio          8544 non-null   object
 13  genre           14414 non-null  object
 14  opening_theme   14478 non-null  object
 15  ending_theme    14478 non-null  object
dtypes: bool(1), object(15)
memory usage: 1.8+ MB


In [9]:
cols = ['title','title_english','title_synonyms', 'type', 'source', 'status','airing',
        'rating', 'background', 'producer', 'licensor', 'studio','genre','opening_theme','ending_theme']

anime['bag_of_words'] = anime[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


In [203]:
for entry in anime.bag_of_words: 
    for word in entry.split(' '):
        if word == 'NaN':
            word == ''
anime.bag_of_words.head(1)[0]

'Inu x Boku SS Inu X Boku Secret Service Youko x Boku SS TV Manga Finished Airing False PG-13 - Teens 13 or older Inu x Boku SS was licensed by Sentai Filmworks for North America, while MVM Films licensed it for the United Kingdom. During April 2013, Hanabee Entertainment released the series on DVD and Blu-ray for Australia and New Zealand. Aniplex, Square Enix, Mainichi Broadcasting System, Movic, Inu x Boku SS Production Partners Sentai Filmworks David Production Comedy, Supernatural, Romance, Shounen [\'"Nirvana" by MUCC\'] [\'#1: "Nirvana" by MUCC (eps 1, 11-12)\', \'#2: "Rakuen no Photograph (楽園のPhotograph)" by Soushi Miketsukami (Yuichi Nakamura) (eps 2, 9)\', \'#3: "Kimi wa (君は)" by Ririchiyo Shirakiin (Rina Hidaka) (eps 3, 8)\', \'#4: "one way" by Banri Watanuki (Takuya Eguchi) & Zange Natsume (Mamoru Miyano) (ep 4)\', \'#5: "SM Hantei Forum (SM判定フォーラム)" by Kagerou Shoukiin (Tomokazu Sugita) (ep 5, 10)\', \'#6: "sweets parade" by Karuta Roromiya (Kana Hanazawa) (ep 6)\', \'#7: 

In [10]:
anime['new'] = anime['bag_of_words']

In [13]:
# cleaning the bag of words

new = []
for entry in anime.new:
    entry = entry.translate(str.maketrans('', '', string.punctuation))
    new.append(entry)
# #     break
anime.new = new
anime.new.head(50)[13]

'Tonari no Kaibutsukun My Little Monster Tonari no Kaibutsukun The Monster Next Door My Neighbor Monsterkun TV Manga Finished Airing False PG13  Teens 13 or older nan TV Tokyo Aniplex Nihon Ad Systems Kodansha NIS America Inc Brain039s Base Slice of Life Comedy Romance School Shoujo QA Recital QA リサイタル by Haruka Tomatsu White Wishes by 9nine'

In [14]:
# adding new stop words
new_stops = list(stopwords.words('english'))
to_add = ['ss', '1','2','3','4','5','6','7','8','9']
for i in to_add:
    new_stops.append(i)
# new_stops

In [15]:
# more cleaning
# lowering
new_low = []
for i in anime.new:
    new_low.append(i.lower())
anime.new = new_low

# word_tokens = word_tokenize(example_sent) 
  
# filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  
# filtered_sentence = [] 
  
# for w in word_tokens: 
#     if w not in stop_words: 
#         filtered_sentence.append(w)
toked = []
for entry in anime.new:
    tokens = wt(entry)
    filtered_sentence = [w for w in tokens if not w in new_stops]
    toked.append(" ".join(filtered_sentence))
anime.new = toked
    
    

anime.new.head(50)[13]




'tonari kaibutsukun little monster tonari kaibutsukun monster next door neighbor monsterkun tv manga finished airing false pg13 teens 13 older nan tv tokyo aniplex nihon ad systems kodansha nis america inc brain039s base slice life comedy romance school shoujo qa recital qa リサイタル haruka tomatsu white wishes 9nine'

In [68]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(anime['new'])

# creating a Series for the anime titles 
indices = pd.Series(anime.title)

In [ ]:
rec = ItemRecommender()

count_df = pd.DataFrame(count_matrix.todense(), index=indices.values)

rec.fit(count_df)


In [ ]:
print(rec.get_recommendations('One Piece'))


## Turn it into a function

In [20]:
def cleany(anime):
    anime = anime[['title','title_english','title_japanese','title_synonyms', 'type', 'source', 'status','airing', 'rating', 'background', 'producer', 'licensor', 'studio','genre','opening_theme','ending_theme']]
    anime = anime.dropna(axis = 0, subset = ['title'])
    cols = ['title','title_english','title_synonyms', 'type', 'source', 'status','airing',
        'rating', 'background', 'producer', 'licensor', 'studio','genre','opening_theme','ending_theme']

    anime['new'] = anime[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    new = []
    for entry in anime.new:
        entry = entry.translate(str.maketrans('', '', string.punctuation))
        new.append(entry)
    # #     break
    anime.new = new
    new_low = []
    for i in anime.new:
        new_low.append(i.lower())
    anime.new = new_low
    toked = []
    for entry in anime.new:
        tokens = wt(entry)
        filtered_sentence = [w for w in tokens if not w in new_stops]
        toked.append(" ".join(filtered_sentence))
    anime.new = toked
    return anime


In [ ]:
def run_rec(anime, anime_title):
    try:
        return rec.get_recommendations(anime_title)
    except:
        title = anime_title.split(" ")
        lst = []
        for i in title:
            i = i.capitalize()
            lst.append(i)
        title = " ".join(lst)
        return rec.get_recommendations(title)
run_rec(anime, 'Princess Tutu')

In [58]:
anime.loc[anime['title'] == 'Princess Tutu']


,title,title_english,title_japanese,title_synonyms,type,source,status,airing,rating,background,producer,licensor,studio,genre,opening_theme,ending_theme,bag_of_words,new
3,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,TV,Original,Finished Airing,False,PG-13 - Teens 13 or older,Princess Tutu aired in two parts. The first pa...,"Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"Comedy, Drama, Magic, Romance, Fantasy","['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ...",Princess Tutu Princess Tutu nan TV Original Fi...,princess tutu princess tutu nan tv original fi...


In [56]:
anime

,title,title_english,title_japanese,title_synonyms,type,source,status,airing,rating,background,producer,licensor,studio,genre,opening_theme,ending_theme,bag_of_words,new
0,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,TV,Manga,Finished Airing,False,PG-13 - Teens 13 or older,Inu x Boku SS was licensed by Sentai Filmworks...,"Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ...",Inu x Boku SS Inu X Boku Secret Service Youko ...,inu x boku inu x boku secret service youko x b...
1,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,TV,Manga,Finished Airing,False,PG-13 - Teens 13 or older,NaN,"TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi...",Seto no Hanayome My Bride is a Mermaid The Inl...,seto hanayome bride mermaid inland sea bride t...
2,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",TV,Manga,Finished Airing,False,PG - Children,NaN,"TV Tokyo, Sotsu",NaN,Satelight,"Comedy, Magic, School, Shoujo","['#1: ""Minna no Tamago (みんなのたまご)"" by Shugo Cha...","['#1: ""Rottara Rottara (ロッタラ ロッタラ)"" by Buono! ...",Shugo Chara!! Doki Shugo Chara!! Doki Shugo Ch...,shugo chara doki shugo chara doki shugo chara ...
3,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,TV,Original,Finished Airing,False,PG-13 - Teens 13 or older,Princess Tutu aired in two parts. The first pa...,"Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"Comedy, Drama, Magic, Romance, Fantasy","['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ...",Princess Tutu Princess Tutu nan TV Original Fi...,princess tutu princess tutu nan tv original fi...
4,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,TV,Manga,Finished Airing,False,PG-13 - Teens 13 or older,NaN,"NHK, Shueisha",NaN,J.C.Staff,"Comedy, Drama, Romance, Shounen","['#1: ""Moshimo no Hanashi (もしもの話)"" by nano.RIP...","['#1: ""Pride on Everyday"" by Sphere (eps 1-13)...",Bakuman. 3rd Season Bakuman. Bakuman Season 3 ...,bakuman 3rd season bakuman bakuman season tv m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14473,Gutchonpa Omoshiro Hanashi,NaN,グッチョンパおもしろ話,NaN,TV,Unknown,Finished Airing,False,G - All Ages,NaN,NaN,NaN,NaN,Kids,[],[],Gutchonpa Omoshiro Hanashi nan nan TV Unknown ...,gutchonpa omoshiro hanashi nan nan tv unknown ...
14474,Geba Geba Shou Time!,NaN,ゲバゲバ笑タイム!,NaN,OVA,Unknown,Finished Airing,False,G - All Ages,NaN,Studio Lotus,NaN,NaN,Comedy,[],[],Geba Geba Shou Time! nan nan OVA Unknown Finis...,geba geba shou time nan nan ova unknown finish...
14475,Godzilla: Hoshi wo Kuu Mono,NaN,GODZILLA -星を喰う者-,"Godzilla Part 3, Godzilla: Eater of Stars",Movie,Other,Not yet aired,False,R - 17+ (violence & profanity),NaN,NaN,NaN,NaN,"Action, Sci-Fi, Adventure, Fantasy",[],[],Godzilla: Hoshi wo Kuu Mono nan Godzilla Part ...,godzilla hoshi wo kuu mono nan godzilla part g...
14476,Nippon Mukashibanashi: Sannen Netarou,NaN,日本昔ばなし 三ねん寝太郎,NaN,OVA,Other,Finished Airing,False,G - All Ages,NaN,NaN,NaN,NaN,"Fantasy, Kids",[],[],Nippon Mukashibanashi: Sannen Netarou nan nan ...,nippon mukashibanashi sannen netarou nan nan o...
